In [1]:
import os
import torch
import numpy as np
from denoisplit.configs.hdn_translation_config import get_config
from denoisplit.nets.model_utils import create_model

/home/ubuntu/miniconda3/envs/Superres/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Create a config object  

In [2]:
config = get_config()

In [3]:
config.data.image_size = 64

In [4]:
config.data.image_size

64

### 2. Instantiate the model

In [5]:
data_mean = data_std = {'target': torch.rand(10, 2)}

model = create_model(
    config, 
    data_mean=data_mean, 
    data_std=data_std, 
    val_idx_manager=None
)

[3, 3] [1, 1]
[3, 3] [1, 1]
[BottomUpLayer] McEnabled:0 
[3, 3] [1, 1]
[BottomUpLayer] McEnabled:0 
[3, 3] [1, 1]
[BottomUpLayer] McEnabled:0 
[3, 3] [1, 1]
[BottomUpLayer] McEnabled:0 
[3, 3] [1, 1]
[3, 3] [1, 1]
[3, 3] [1, 1]
[TopDownLayer] normalize_latent_factor:1.0
[3, 3] [1, 1]
[3, 3] [1, 1]
[3, 3] [1, 1]
[TopDownLayer] normalize_latent_factor:1.0
[3, 3] [1, 1]
[3, 3] [1, 1]
[3, 3] [1, 1]
[TopDownLayer] normalize_latent_factor:1.0
[3, 3] [1, 1]
[TopDownLayer] normalize_latent_factor:1.0
[3, 3] [1, 1]
[GaussianLikelihood] PredLVar:None LowBLVar:-5
[LadderVAETranslation] Stoc:True RecMode:False TethInput:False


In [17]:
from torchinfo import summary
bs = 32
summary(model, input_size=(bs, 1, 64, 64))

Layer (type:depth-idx)                                  Output Shape              Param #
LadderVAETranslation                                    [32, 1, 64, 64]           --
├─Sequential: 1-1                                       [32, 64, 64, 64]          --
│    └─Conv2d: 2-1                                      [32, 64, 64, 64]          640
│    └─ELU: 2-2                                         [32, 64, 64, 64]          --
│    └─BottomUpDeterministicResBlock: 2-3               [32, 64, 64, 64]          --
│    │    └─ResidualBlock: 3-1                          [32, 64, 64, 64]          74,112
├─ModuleList: 1-2                                       --                        --
│    └─BottomUpLayer: 2-4                               [32, 64, 32, 32]          --
│    │    └─Sequential: 3-2                             [32, 64, 32, 32]          119,360
│    │    └─Sequential: 3-3                             [32, 64, 32, 32]          --
│    └─BottomUpLayer: 2-5                         

In [7]:
first_bottom_up_block = model.first_bottom_up
summary(first_bottom_up_block, input_size=(32, 1, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 64, 64, 64]          --
├─Conv2d: 1-1                            [32, 64, 64, 64]          640
├─ELU: 1-2                               [32, 64, 64, 64]          --
├─BottomUpDeterministicResBlock: 1-3     [32, 64, 64, 64]          --
│    └─ResidualBlock: 2-1                [32, 64, 64, 64]          --
│    │    └─Sequential: 3-1              [32, 64, 64, 64]          74,112
Total params: 74,752
Trainable params: 74,752
Non-trainable params: 0
Total mult-adds (G): 9.76
Input size (MB): 0.52
Forward/backward pass size (MB): 335.54
Params size (MB): 0.30
Estimated Total Size (MB): 336.37

In [8]:
bottom_up_block = model.bottom_up_layers[0]

In [9]:
summary(bottom_up_block, input_size=(32, 64, 32, 32))

Layer (type:depth-idx)                        Output Shape              Param #
BottomUpLayer                                 [32, 64, 16, 16]          --
├─Sequential: 1-1                             [32, 64, 16, 16]          --
│    └─BottomUpDeterministicResBlock: 2-1     [32, 64, 16, 16]          --
│    │    └─Conv2d: 3-1                       [32, 64, 16, 16]          36,928
│    │    └─ResidualBlock: 3-2                [32, 64, 16, 16]          82,432
├─Sequential: 1-2                             [32, 64, 16, 16]          --
Total params: 119,360
Trainable params: 119,360
Non-trainable params: 0
Total mult-adds (M): 975.71
Input size (MB): 8.39
Forward/backward pass size (MB): 29.36
Params size (MB): 0.48
Estimated Total Size (MB): 38.23

In [21]:
top_down_layer = model.top_down_layers[1]

In [ ]:
summary(top_down_layer, input_size=(32, 64, 8, 8))

Checking how the function `get_reconstruction_loss()` works 

In [13]:
from denoisplit.nets.lvae import LadderVAE

bs = 32
h = w = 64
img_shape = (bs, 1, h, w) 
lowres_in = torch.rand(img_shape)
highres_gt = torch.rand(img_shape)
model_out = torch.rand(img_shape)
mask = ~((highres_gt == 0).reshape(len(highres_gt), -1).all(dim=1))

(highres_gt == 0).reshape(len(highres_gt), -1)

# model.get_reconstruction_loss(reconstruction=model_out, target=highres_gt, input=lowres_in, )

32